In [ ]:
using LibPQ, Tables, Plots, HTTP, CSV, DataFrames, Random, StatsBase, FreqTables, SQLite, Base, Dates
using Base: show_supertypes
plotly()


In [ ]:
import Pkg; Pkg.add("Date")

In [ ]:
conn = LibPQ.Connection("dbname=sharath")
sqlite = SQLite.DB()

In [ ]:
# result = execute(conn, """select "NDP_Date" , "NDP_Close" from ace_equity_nifty_50 aen where "Company Name" = 'Infosys Ltd.' order by 1""")
# result = execute(conn, """select "NDP_Date" , "NDP_Close" from ace_equity_nifty_50 aen where "Company Name" = 'Adani Ports and Special Economic Zone Ltd.' order by 1""")
result = execute(conn, """select date, adjclose from nse_raw nr where symbol = 'ANANTRAJ' and adjclose is not null order by 1 """)

data = columntable(result)
series = copy(data[2])
data

In [ ]:
typeof(data)

In [ ]:
# print(rand(length(x)))
plot((data[1]), copy(data[2]))

In [ ]:
function sip(sipFrequencyDays::Int, sipDurationDays::Int, series)
    stocks::Float64 = 0.0
    n = size(series)[1]
    rates = Vector{Float64}(undef, n-sipDurationDays+1)
    for i in 1:n-sipDurationDays+1
        innerRange = i:sipFrequencyDays:i+sipDurationDays-1
        for j in innerRange
            stocks = stocks + 1/series[j]
        end
        rates[i] = r(lastindex(innerRange), series[last(innerRange)]*stocks, sipDurationDays/365.0)*100
    end
    return nquantile(rates, 100)
end
# println(typeof(returns[1, :]))
function go(series)
    years = 1:6
    returns = Array{Float64}(undef, 101, length(years))

    for year in years
        returns[:, year] = sip(1, (year+1)*365, series)
    end
    return returns
end
plotData = @time go(series)

# print(length(data[2]))
# print(1/data.NDP_Close[3])
# innerRange = 1:3:45
# collect(last(innerRange))

In [ ]:
length(1:6)
plot(x, plotData, label = permutedims(["year"*string(x) for x in 2:7]))

# print(rand(2, 2))

In [ ]:
show_supertypes(typeof(data.NDP_Date))

In [ ]:
function ohlc2()
    function get_ohlc(ticker; kwargs...)
        res = HTTP.get("https://www.quandl.com/api/v3/datasets/WIKI/$(ticker)/data.csv?start_date=2017-01-01")
        df = CSV.read(res.body)
        ohlc(df, x=:Date, open=:Open, high=:High, low=:Low, close=:Close; kwargs...)
    end

    p1 = plot(get_ohlc("AAPL", name="Apple"), Layout(title="Apple"))
    p2 = plot(get_ohlc("GOOG", name="Google"), Layout(title="Google"))

    [p1 p2]
end
ohlc2()

In [ ]:
function two_hists()
    x0 = randn(500)
    x1 = x0 .+ 1

    trace1 = histogram(x=x0, opacity=0.75)
    trace2 = histogram(x=x1, opacity=0.75)
    data = [trace1, trace2]
    layout = Layout(barmode="overlay")
    plot(data, layout)
end
two_hists()

In [ ]:
function go(r::Number)
    [x for x in 1:5]
end
go(0.07)

In [ ]:
death = 100
age = 37
months = (death - age)*12
inflationPerMonth = 6/(100*12)
monthlyCost = 100000
living = monthlyCost * months
println(living)
marriage = 1000000
# education is 100000 per year (present value) starting from 5th year to 5+16 years

education = 16*100000
total = living + marriage + 2*education
println(total)

In [ ]:
function p2(p1::Number, r::Number, t::Number)
    return Float64(p1)*((1+Float64(r))^Float64(t))
end

function t(p1::Number, p2::Number, r::Number)
    return log(Float64(p2)/Float64(p1)) / log(1+Float64(r))
end

function p1(p2::Number, r::Number, t::Number)
    return Float64(p2)/((1+Float64(r))^Float64(t))
end

function r(p1::Number, p2::Number, t::Number)
    power = log(Float64(p2)/Float64(p1))/Float64(t)
    return exp(power) - 1
end

function emi(p1::Number, r::Number, t::Number)
    x::Float64 = (1.0 + Float64(r))^Float64(t)
    return Float64(p1) * Float64(r) * x / (x - 1)
end


In [ ]:
df = CSV.read("/home/sharath/Downloads/AdultDataSet.csv")

In [ ]:
df2 = df[shuffle(1:nrow(df))[1:Int(floor((nrow(df)/5)))], :]

In [ ]:
function two_hists(df, df2)
    trace1 = histogram(x=df["age"], opacity=0.75)
    trace2 = histogram(x=df2["age"], opacity=0.75)
    data = [trace1, trace2]
    layout = Layout(barmode="overlay")
    plot(data, layout)
end
two_hists(df, df2)

In [ ]:
plot([histogram(x=randn(10000), opacity=0.75)], Layout(barmode="overlay"))



In [ ]:
filter(row->row["education"]==" HS-grad" && row["marital-status"]==" Never-married", df)

In [ ]:
typeof(freqtable(df["education"], df["marital-status"]))

In [ ]:
df |> SQLite.load!(sqlite, "adult")

In [ ]:
df = DBInterface.execute(sqlite, "select * from adult") |> DataFrame